In [ ]:
from functions import *

# greeks("NVDA", '2026-01-16', 175, "put")

In [ ]:
continuous_monitor("AMC")

In [ ]:
greeks("AMC", '2026-01-16', 2, "call")

In [ ]:
option_price("AMC", '2026-01-16', 2, "call")

In [ ]:
# Get historical stock price
price = get_stock_price_historical('AAPL', '2024-01-15')
print(f"AAPL price on 2024-01-15: ${price}")

# Calculate theoretical option price on a historical date
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Theoretical call price: ${option_price}")

# Calculate Greeks on a historical date
greeks_data = greeks_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Historical Greeks: {greeks_data}")

# With custom implied volatility
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15', iv=0.25
)
print(f"Price with 25% IV: ${option_price}")


In [ ]:
# Get intraday stock price
price = get_stock_price_intraday('AAPL', '2025-12-01', '14:30')
print(f"AAPL at 2:30 PM: ${price}")

# Calculate option price at a specific time
option_price = option_price_intraday(
    'AAPL', '2026-01-16', 150.0, 'call', 
    '2025-12-01', '14:30'
)
print(f"Call option price at 2:30 PM: ${option_price}")

# Calculate Greeks at a specific time
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30'
)
print(f"Greeks at 2:30 PM: \n{greeks}")

# With custom IV
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30', iv=0.28
)
print(f"Greeks at 2:30 PM with IV = 28%: \n{greeks}")

In [ ]:
from functions import *

# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AAPL', 
    '2026-01-16',  # Expiration
    150.0,          # Strike
    '2025-12-10',   # Purchase date
    '10:30',        # Purchase time (10:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


In [ ]:
from functions import *

# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AMC', 
    '2026-01-16',  # Expiration
    2,          # Strike
    '2025-12-16',   # Purchase date
    '09:30',        # Purchase time (09:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


In [2]:
from functions import *

df = pd.read_csv('nasdaq_tickers.csv')

reasonable_stocks = []

for n in range(0,600):
    ticker, price = list(df[['Ticker','Last Price']].iloc[n])
    if 3*price < 100:
        reasonable_stocks.append([ticker, price])

reasonable_stocks = [[ticker, float(price)] for ticker, price in reasonable_stocks]

In [3]:
reasonable_stocks[0]

def check_option(ticker, price, date = '2026-01-16', margin = 0.01):
    try:
        # Get the option chain to see available strikes
        ticker_obj = yf.Ticker(ticker)
        chain = ticker_obj.option_chain(date)
        
        # Check if option chain has data
        if chain.calls.empty or chain.puts.empty:
            print(f"{ticker} - No option data available")
            return None
        
        # Find nearest strike for call (above current price)
        call_strikes = chain.calls['strike'].values
        call_target = price * (1 + margin)
        call_strike = call_strikes[np.argmin(np.abs(call_strikes - call_target))]
        #print(f"Call target: {call_target}, selected strike: {call_strike}")    

        # Find nearest strike for put (below current price)
        put_strikes = chain.puts['strike'].values
        put_target = price * (1 - margin)
        put_strike = put_strikes[np.argmin(np.abs(put_strikes - put_target))]
        #print(f"Put target: {put_target}, selected strike: {put_strike}")
        
        call_value = option_price(ticker, date, call_strike, 'call')
        call_value['strike'] = float(call_strike)
        call_value['ticker'] = ticker
        call_value['date'] = date

        put_value = option_price(ticker, date, put_strike, 'put')
        put_value['strike'] = float(put_strike)
        put_value['ticker'] = ticker
        put_value['date'] = date

        #print(f"{ticker} (${price:.2f})\n  Call ${call_strike}: {call_value}\n  Put ${put_strike}: {put_value}")
        return {'call': call_value  , 'put': put_value}
    
    except Exception as e:
        print(f"{ticker} - Error: {type(e).__name__}: {e}")
        return None

In [4]:
check_option("CMCSA", 30)['put']

{'lastPrice': 1.07,
 'bid': 1.01,
 'ask': 1.08,
 'volume': 244,
 'strike': 30.0,
 'ticker': 'CMCSA',
 'date': '2026-01-16'}

In [5]:
greeks("CMCSA", '2026-01-16', 30, "put", status=True, silent = True)

{'delta': -0.43576401691497113,
 'gamma': 0.13493620995308409,
 'vega': 0.033568911075165804,
 'theta': -0.018434552305614024,
 'rho': -0.01126616464617982,
 'buy_score': 72,
 'recommendation': 'BUY'}

In [ ]:
for ticker, price in reasonable_stocks: #197 entries
    print(f"Analyzing {ticker} at price ${price}")
    info = check_option(ticker, price)
    if info is None:
        continue
    print(f'Option contract price is approximately ${info["call"]["lastPrice"]*100}')
    print("")

    put_strike = info['put']['strike']
    call_strike = info['call']['strike']
    put_buy_score = greeks(ticker, '2026-01-16', info['put']['strike'], "put", status=True, silent = True)['buy_score']
    call_buy_score = greeks(ticker, '2026-01-16', info['call']['strike'], "call", status=True, silent = True)['buy_score']
    
    print(f'Buy score for PUT option at {put_strike} on 2026-01-16: {put_buy_score}%')
    print(f'Buy score for CALL option at {call_strike} on 2026-01-16: {call_buy_score}%')

    print('')
    print("-------------------------------------------------------")
    print('')


Analyzing CMCSA at price $29.73
Option contract price is approximately $112.00000000000001

Buy scores for options expiring on 2026-01-16:
Buy score for PUT option at 30.0 on 2026-01-16: 72%
Buy score for CALL option at 30.0 on 2026-01-16: 82%
-------------------------------------------------------
Analyzing NU at price $16.2
Option contract price is approximately $55.00000000000001

Buy scores for options expiring on 2026-01-16:
Buy score for PUT option at 16.0 on 2026-01-16: 85%
Buy score for CALL option at 16.0 on 2026-01-16: 82%
-------------------------------------------------------
Analyzing KDP at price $28.59
Option contract price is approximately $42.0

Buy scores for options expiring on 2026-01-16:
Buy score for PUT option at 28.0 on 2026-01-16: 77%
Buy score for CALL option at 29.0 on 2026-01-16: 67%
-------------------------------------------------------
Analyzing SOFI at price $26.58
Option contract price is approximately $119.0

Buy scores for options expiring on 2026-01-

KeyboardInterrupt: 